In [1]:
# https://leetcode.cn/problems/article-views-i/description/
# [去重distinct、排序order by、重新命名rename as]
# 請查詢出所有瀏覽過自己文章的作者。結果按照作者的 id 升序排列。

In [ ]:
import pandas as pd
import pymysql 

host = 'localhost' 
port = 3306    
user = 'root'  

passwd = 'password'   
db = 'testdb'          
charset = 'utf8mb4'     

conn = pymysql.connect(host=host, port=port, user=user, passwd=passwd, db=db, charset=charset)

print('Successfully connected!')

Successfully connected!


In [3]:
# 創建游標
cursor = conn.cursor() 

In [4]:
# 使用sql語法創建表格
sql_create= """
CREATE TABLE IF NOT EXISTS Views (
    article_id INT,
    author_id INT,
    viewer_id INT,
    view_date DATE
);
"""
# 執行建表指令
cursor.execute(sql_create)
print("成功建立 Views 資料表")

成功建立 Views 資料表


In [5]:
sql_insert= """
INSERT INTO Views (article_id, author_id, viewer_id, view_date) VALUES
(1, 3, 5, '2019-08-01'),
(1, 3, 6, '2019-08-02'),
(2, 7, 7, '2019-08-01'),
(2, 7, 6, '2019-08-02'),
(4, 7, 1, '2019-07-22'),
(3, 4, 4, '2019-07-21'),
(3, 4, 4, '2019-07-21');
"""

# 執行寫入資料指令
cursor.execute(sql_insert)
print("成功將資料寫入 Views 資料表")

成功將資料寫入 Views 資料表


In [6]:
conn.commit()
#哪些行為要特別committ？
# 1. 插入資料：insert 
# 2. 更新資料：update、delete
# 為何truncate不需要committ？
# 因為 truncate 是直接刪除整個表格的資料，不需要提交，它是在重新定義一個新的資源，(一開始的drop也是一種完整的定義行為)、(delete是刪除部分資料的行為)

In [7]:
# #如果表格已經存在，是要插入columns，可用以下方法：
# # 1.先寫好 SQL 語法 SQL的columns模板、並將語法中會不斷改變的部分挖空 ( %s )
# sql_insert = """
# INSERT INTO Staff (ID, Name, DeptId, Age, Gender, Salary, recordDt)
# VALUES (%s, %s, %s, %s, %s, %s, %s);
# """

# # 2.整理 jsondata 成可 insert 進資料庫的樣子，格式如下(其中[]裡面的()是tuple的格式)：
# values = [
#     ('001', 'Mike', '002', '45', 'M', '60000', '2020-10-13 21:41:49'),
#     ('002', 'Judy', '002', '30', 'F', '48000', '2020-10-13 21:41:49'),
#     ('003', 'Allen', '001', '22', 'M', '50000', '2020-10-13 21:41:49'),
#     ('004', 'Tom', '002', '47', 'M', '47000', '2020-10-13 21:41:49'),
#     ('005', 'Jack', '003', '36', 'M', '52000', '2020-10-13 21:41:49'),
#     ('006', 'Abby', '002', '24', 'F', '45000', '2020-10-13 21:41:49'),
#     ('007', 'Trump', '001', '80', 'M', '80000', '2020-10-13 21:41:49'),
#     ('008', 'Marry', '003', '29', 'F', '87000', '2020-10-13 21:41:49')
# ]

# # 3.使用 cursor.executemany() 方法批量插入資料，(插入的mysql模板,插入的資料)
# cursor.executemany(sql_insert, values)

# # 4.提交資料
# conn.commit() 

In [8]:
#接下來要用mysql語法 select資料出來看看：
sql_look = """
select * from Views
"""
cursor.execute(sql_look)
#回應1代表裡面有一筆資料

42

In [9]:
cursor.scroll(0, mode='absolute')
df=pd.DataFrame(cursor.fetchall())
print(df)

    0  1  2           3
0   1  3  5  2019-08-01
1   1  3  6  2019-08-02
2   2  7  7  2019-08-01
3   2  7  6  2019-08-02
4   4  7  1  2019-07-22
5   3  4  4  2019-07-21
6   3  4  4  2019-07-21
7   1  3  5  2019-08-01
8   1  3  6  2019-08-02
9   2  7  7  2019-08-01
10  2  7  6  2019-08-02
11  4  7  1  2019-07-22
12  3  4  4  2019-07-21
13  3  4  4  2019-07-21
14  1  3  5  2019-08-01
15  1  3  6  2019-08-02
16  2  7  7  2019-08-01
17  2  7  6  2019-08-02
18  4  7  1  2019-07-22
19  3  4  4  2019-07-21
20  3  4  4  2019-07-21
21  1  3  5  2019-08-01
22  1  3  6  2019-08-02
23  2  7  7  2019-08-01
24  2  7  6  2019-08-02
25  4  7  1  2019-07-22
26  3  4  4  2019-07-21
27  3  4  4  2019-07-21
28  1  3  5  2019-08-01
29  1  3  6  2019-08-02
30  2  7  7  2019-08-01
31  2  7  6  2019-08-02
32  4  7  1  2019-07-22
33  3  4  4  2019-07-21
34  3  4  4  2019-07-21
35  1  3  5  2019-08-01
36  1  3  6  2019-08-02
37  2  7  7  2019-08-01
38  2  7  6  2019-08-02
39  4  7  1  2019-07-22
40  3  4  4  201

In [10]:
# 如果用 fetchall() 直接轉成 DataFrame，預設會以數字當欄名，除非手動設定 columns=...
# cursor.description 是一個 tuple，裡面包含每個欄位的資訊，要的是欄位名稱也就是desc[0]
print(cursor.description)

(('article_id', 3, None, 11, 11, 0, True), ('author_id', 3, None, 11, 11, 0, True), ('viewer_id', 3, None, 11, 11, 0, True), ('view_date', 10, None, 10, 10, 0, True))


In [11]:
cursor.execute("SELECT * FROM Views")
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]

df = pd.DataFrame(rows, columns=columns)
print(df)

    article_id  author_id  viewer_id   view_date
0            1          3          5  2019-08-01
1            1          3          6  2019-08-02
2            2          7          7  2019-08-01
3            2          7          6  2019-08-02
4            4          7          1  2019-07-22
5            3          4          4  2019-07-21
6            3          4          4  2019-07-21
7            1          3          5  2019-08-01
8            1          3          6  2019-08-02
9            2          7          7  2019-08-01
10           2          7          6  2019-08-02
11           4          7          1  2019-07-22
12           3          4          4  2019-07-21
13           3          4          4  2019-07-21
14           1          3          5  2019-08-01
15           1          3          6  2019-08-02
16           2          7          7  2019-08-01
17           2          7          6  2019-08-02
18           4          7          1  2019-07-22
19           3      

In [12]:
# 用python語法執行
def article_views(views: pd.DataFrame) -> pd.DataFrame:
    return(
        views.loc[
            (views["author_id"]==views["viewer_id"],["author_id"])
        ]
        .rename(columns={"author_id":"id"})
        .drop_duplicates()
        .sort_values(['id'])
    )

article_views(df)

,id
5,4
2,7


In [13]:
# 用Mysql語法執行
sql_article_views="""
select distinct author_id as id 
from Views 
where author_id=viewer_id 
order by id;
"""

cursor.execute(sql_article_views)
columns = [desc[0] for desc in cursor.description]
dk = pd.DataFrame(cursor.fetchall(), columns=columns)
print(dk)

# 不能直接引用上面的df嗎?
# 不行，在每次 cursor.execute(...) 之後，都要重新取出結果與欄位名稱，然後再建立 DataFrame
# cursor.execute() 會覆蓋先前的查詢結果
# cursor.fetchall() 只能抓最新的查詢資料
# cursor.description 也是針對最新的 SQL 查詢


   id
0   4
1   7


In [14]:
#但是可以寫一個通用函數：
def fetch_to_df(cursor, sql):
    cursor.execute(sql)
    return pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

df1 = fetch_to_df(cursor, "SELECT * FROM Views")
df2 = fetch_to_df(cursor, """
    select distinct author_id as id 
    from Views 
    where author_id = viewer_id 
    order by id;
""")

print(df1)

    article_id  author_id  viewer_id   view_date
0            1          3          5  2019-08-01
1            1          3          6  2019-08-02
2            2          7          7  2019-08-01
3            2          7          6  2019-08-02
4            4          7          1  2019-07-22
5            3          4          4  2019-07-21
6            3          4          4  2019-07-21
7            1          3          5  2019-08-01
8            1          3          6  2019-08-02
9            2          7          7  2019-08-01
10           2          7          6  2019-08-02
11           4          7          1  2019-07-22
12           3          4          4  2019-07-21
13           3          4          4  2019-07-21
14           1          3          5  2019-08-01
15           1          3          6  2019-08-02
16           2          7          7  2019-08-01
17           2          7          6  2019-08-02
18           4          7          1  2019-07-22
19           3      

In [15]:
print(df2)

   id
0   4
1   7


In [16]:
#將游標與連結關閉
cursor.close()
conn.close()